In [1]:
from disk_analysis_tools import tiling_disk_plots as tdp
from disk_analysis_tools import tiling_disk_utils as tdu

import pandas as pd 
import sqlite3 as sql

google sheet for db

In [2]:
sheet_id = '1Pz2pHkf7AzKAfGPPVjLGofgpcrwVyGpTJu8phObsqvg'
sheet_name = 'measurements_meta_db'
exp_name = 'experiments_db'

meta_url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'
exp_url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={exp_name}'

In [3]:
meta_data = pd.read_csv(meta_url)
exp_data = pd.read_csv(exp_url)

In [23]:
meta_data.tail(5)

,measurement_id,exp_id,date,file_name,material,process_step,vac_mapping,coordinates,meas_cap_status,comments
133,134.0,26.0,21.04.2022,20220421_steeplate_01_curing_bfvacr.txt,steel,before_glue,"[1,60]",coordinate-top.txt,1.0,first measurement instantly after gluing (vac ...
134,135.0,26.0,21.04.2022,20220421_steeplate_01_curing.txt,steel,curing,"[n,1000]",coordinate-top.txt,1.0,first measurement instantly after gluing (vac ...
135,136.0,27.0,28.04.2022,20220428_trip_v8_ref_vac.txt,steel,before_glue,"[1,60]",coordinate-blue-brown-yellow.txt,1.0,NaN
136,137.0,26.0,02.05.2022,20220502_table_ref.txt,table,experiment,"[1,1000]",coordinate-top.txt,1.0,ref for measuring plate again after handeling
137,138.0,26.0,02.05.2022,20220502_steelplate_01_after_handling.txt,steel,curing,"[1,1000]",coordinate-top.txt,1.0,ref measurement after handling the plate for a...


In [24]:
exp_data.tail(5)

,exp_id,exp_description,exp_description_short
22,23,triplets v5: blue: v=3.3mm/s; brown:v=3.8mm/s;...,"triplets v5 v=[3-4], blue tip"
23,24,triplets v6: blue: v=3.3mm/s; brown:v=3.8mm/s;...,"triplets v6 v=[3-4.5], blue tip"
24,25,triplets v7: blue: v=3.8mm/s; brown:v=4mm/s; v...,"triplets v7,v=[3-4.2], blue tip"
25,26,steeldisk 01: first full disk,Steel disk 01
26,27,"triplets v8: curing strategy, triplets blue:re...",triplets v8


In [4]:
meta_data = meta_data.drop(columns='meas_short_label')
meta_data = meta_data.iloc[0:138,:]

In [21]:
meta_data.tail(2)

,measurement_id,exp_id,date,file_name,material,process_step,vac_mapping,coordinates,meas_cap_status,comments
136,137.0,26.0,02.05.2022,20220502_table_ref.txt,table,experiment,"[1,1000]",coordinate-top.txt,1.0,ref for measuring plate again after handeling
137,138.0,26.0,02.05.2022,20220502_steelplate_01_after_handling.txt,steel,curing,"[1,1000]",coordinate-top.txt,1.0,ref measurement after handling the plate for a...


manual check where we have nulls

In [5]:
exp_data = exp_data.iloc[:27,:]

In [22]:
exp_data.tail(3)

,exp_id,exp_description,exp_description_short
24,25,triplets v7: blue: v=3.8mm/s; brown:v=4mm/s; v...,"triplets v7,v=[3-4.2], blue tip"
25,26,steeldisk 01: first full disk,Steel disk 01
26,27,"triplets v8: curing strategy, triplets blue:re...",triplets v8


In [9]:
for row in meta_data.iterrows(): 
    if row[0] == 0:
        print(row[1].keys())


Index(['measurement_id', 'exp_id', 'date', 'file_name', 'exp_description',
       'material', 'process_step', 'vac_mapping', 'coordinates',
       'meas_cap_status', 'comments'],
      dtype='object')


In [10]:
def placeholders(len): 
    def flatten(A):
    # generic stolen flatten function from stackoverflow
        rt = []
        for i in A:
            if isinstance(i,list): rt.extend(flatten(i))
            else: rt.append(i)
        return rt
    s = list('(')
    s.append(['?' for x in range(len)])
    s.append(')')
    s = flatten(s)
    s = str(s)
    s = s.replace("'","")
    s = s.replace("[(,",'(')
    s = s.replace(", )]",")")
    return s

In [11]:
placeholders(5)

'( ?, ?, ?, ?, ?)'

In [12]:

command = f"""insert into a
                values {placeholders(5)}"""
command

'insert into a\n                values ( ?, ?, ?, ?, ?)'

In [13]:
def insert_values(row, table): 
    row_series = row[1]
    row_dict = row_series.to_dict()
    columns = tuple([f':{x}' for x in row_dict.keys()])

    # print(len(columns))
    # print(tuple(row_dict.values()))
    # print(str(row_dict))
    
    command = f"""insert into {str(table)}
                    values {placeholders(len(columns))}"""
    print(command)
    conn.execute(command, tuple(row_dict.values()))

create experiments table

In [14]:
# with sql.connect('measurements.db') as conn: 
#     conn.execute("""create table experiments (
#         exp_id int ,
#         exp_description varchar(1000),
#         exp_description_short varchar(100),
#         PRIMARY KEY(exp_id)
#         )"""
#     )


populate experiments table

In [15]:
# with sql.connect('measurements.db') as conn: 
#     for row in exp_data.iterrows(): 
#         insert_values(row, 'experiments')

insert into experiments
                    values ( ?, ?, ?)
insert into experiments
                    values ( ?, ?, ?)
insert into experiments
                    values ( ?, ?, ?)
insert into experiments
                    values ( ?, ?, ?)
insert into experiments
                    values ( ?, ?, ?)
insert into experiments
                    values ( ?, ?, ?)
insert into experiments
                    values ( ?, ?, ?)
insert into experiments
                    values ( ?, ?, ?)
insert into experiments
                    values ( ?, ?, ?)
insert into experiments
                    values ( ?, ?, ?)
insert into experiments
                    values ( ?, ?, ?)
insert into experiments
                    values ( ?, ?, ?)
insert into experiments
                    values ( ?, ?, ?)
insert into experiments
                    values ( ?, ?, ?)
insert into experiments
                    values ( ?, ?, ?)
insert into experiments
                    values ( ?, ?, ?)
insert i

In [19]:
with sql.connect('measurements.db') as conn: 
    conn.execute("""create table measurement_meta (
        measurement_id int,
        exp_id int ,
        file_name varchar(50),
        date datetime,
        material varchar(50), 
        process_step varchar(50), 
        vac_mapping varchar(50), 
        coordinates varchar(50),
        meas_cap_status bool,
        comments varchar(1000),
        PRIMARY KEY (measurement_id),
        FOREIGN KEY(exp_id) REFERENCES experiments(exp_id)
        );"""
    )


In [17]:
meta_data.keys()

Index(['measurement_id', 'exp_id', 'date', 'file_name', 'exp_description',
       'material', 'process_step', 'vac_mapping', 'coordinates',
       'meas_cap_status', 'comments'],
      dtype='object')

In [18]:
meta_data = meta_data.drop(columns='exp_description')

In [125]:
# meta_data = meta_data.drop(columns=['file_name', 'exp_description'])

In [20]:
with sql.connect('measurements.db') as conn: 
    for row in meta_data.iterrows(): 
        insert_values(row, 'measurement_meta')

insert into measurement_meta
                    values ( ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
insert into measurement_meta
                    values ( ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
insert into measurement_meta
                    values ( ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
insert into measurement_meta
                    values ( ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
insert into measurement_meta
                    values ( ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
insert into measurement_meta
                    values ( ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
insert into measurement_meta
                    values ( ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
insert into measurement_meta
                    values ( ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
insert into measurement_meta
                    values ( ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
insert into measurement_meta
                    values ( ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
insert into measurement_meta
                    values ( ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
insert into measurement_meta
   

# query the above created db

In [25]:
def query_exp_db(db_path='measurements.db'):
    '''retrieve experiments descriptions from measurements.db'''
    import sqlite3
    import pandas as pd
    with sql.connect(db_path) as conn: 
        c = conn.cursor()
        
        #* Query SQL data
        sql_query = f"""SELECT  *
        from experiments """

        c.execute(sql_query)
        t = c.fetchall()
        
    #* Read into dataframe
    exp_df = pd.DataFrame(t,
                        columns=["exp_id",
                                "exp_description",
                                "exp_description_short"])
    return exp_df

In [66]:
def fetch_meas_metadata(exp_id=1, db_path='measurements.db'): 
    '''retrieve experiments descriptions from measurements.db'''
    import sqlite3
    import pandas as pd
    with sql.connect(db_path) as conn: 
        c = conn.cursor()
        
        #* Query SQL data
        sql_query = f"""SELECT  *
        from measurement_meta WHERE exp_id == {exp_id} """

        c.execute(sql_query)
        meas_meta = c.fetchall()

    #* Read into dataframe
    meta_df = pd.DataFrame(meas_meta,
                        columns=['measurement_id',
                                'exp_id',
                                'file_name',
                                'date',
                                'material', 
                                'process_step', 
                                'vac_mapping', 
                                'coordinates',
                                'meas_cap_status',
                                'comments'
                                ])
    return meta_df

In [67]:
df_test = fetch_meas_metadata(exp_id=1)

In [68]:
df_test

,measurement_id,exp_id,file_name,date,material,process_step,vac_mapping,coordinates,meas_cap_status,comments
0,1,1,06.09.2021,MyScan_Test_1Tile.txt,table,experiment,"[1,1000]",coordinate-tile1.txt,0,placed by hand; z_offset=158.28
1,2,1,06.09.2021,MyScan_Test_1Tile_LAO1_noVac.txt,lao,experiment,"[1,1000]",coordinate-tile1.txt,0,placed by hand;orientation: matte; z_offset=157.3
2,3,1,06.09.2021,MyScan_Test_1Tile_LAO1_Vac.txt,lao,experiment,"[1,60]",coordinate-tile1.txt,0,placed by hand;orientation: matte; z_offset=157.3
3,4,1,06.09.2021,MyScan_Test_1Tile_LAO1_Vac_pol.txt,lao,experiment,"[1,1000]",coordinate-tile1.txt,0,polished side is highly reflective!; placed by...
4,5,1,06.09.2021,SteelScan_notiles.txt,table,experiment,"[1,1000]",None,0,placed by hand; z_offset=158.26
5,6,1,07.09.2021,SteelScan_points_up.txt,steel,experiment,"[1,60]",None,0,Not positioned accurate!; Fingerprints on this...
6,7,1,07.09.2021,SteelScan_points_down.txt,steel,experiment,"[1,60]",None,0,Not positioned accurate!; Gloves present now; ...
